[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ElMartinez31/Data_Science/blob/main/Projects/Classical_Frameworks/Text_Rank.ipynb)

In [21]:
import pandas as pd
import numpy as np
import textwrap
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
# we use the same dataset as for the previous project
df = pd.read_csv("/content/drive/Othercomputers/Mon ordinateur portable/Data Science/Text_Summary/df_total.csv")

In [24]:
df.head()

,url,news,Type
0,https://www.larepublica.co/redirect/post/3201905,Durante el foro La banca articulador empresari...,Otra
1,https://www.larepublica.co/redirect/post/3210288,El regulador de valores de China dijo el domin...,Regulaciones
2,https://www.larepublica.co/redirect/post/3240676,En una industria históricamente masculina como...,Alianzas
3,https://www.larepublica.co/redirect/post/3342889,Con el dato de marzo el IPC interanual encaden...,Macroeconomia
4,https://www.larepublica.co/redirect/post/3427208,Ayer en Cartagena se dio inicio a la versión n...,Otra


In [25]:
#same idea we want to get one document to process it

doc = df['news'].sample().iloc[0]


In [26]:
print(doc)

La prolongación de la vida laboral mejora la sostenibilidad del estado de bienestar y del sistema público de pensiones, tiene externalidades positivas sobre el empleo y la productividad de los trabajadores jóvenes y favorece el funcionamiento cognitivo. Estas son las principales conclusiones del informe de BBVA Research ‘Prolongar la vida laboral: ¿Por qué? ¿Dónde estamos? ¿Cómo hacerlo?’, que analiza la evolución en la tasa de empleo de los mayores de 55 años y explica medidas de éxito para impulsar la empleabilidad de este colectivo.
La tasa de actividad de la población española entre 55 y 64 años se ha incrementado más de 25 puntos desde mediados de la década de los 90, pasando de un 37% en 1995 hasta el 63% de 2020. Un dato nada desdeñable pero todavía lejos de las tasas de países como Suecia (83%), Estonia, Nueva Zelanda o Japón (79%). Una diferencia que se irá acortando según la Comisión Europea, que estima que, con las políticas adecuadas, la participación laboral de la població

In [27]:
# we can see that the text is 'cut' at end of some sentences.
# We will use the same method as last project to correct this

def wrap(x):
  return textwrap.fill(x, replace_whitespace=False, fix_sentence_endings= True)

In [28]:
print(wrap(doc)) #OK

La prolongación de la vida laboral mejora la sostenibilidad del estado
de bienestar y del sistema público de pensiones, tiene externalidades
positivas sobre el empleo y la productividad de los trabajadores
jóvenes y favorece el funcionamiento cognitivo.  Estas son las
principales conclusiones del informe de BBVA Research ‘Prolongar la
vida laboral: ¿Por qué? ¿Dónde estamos?  ¿Cómo hacerlo?’, que analiza
la evolución en la tasa de empleo de los mayores de 55 años y explica
medidas de éxito para impulsar la empleabilidad de este colectivo.
La
tasa de actividad de la población española entre 55 y 64 años se ha
incrementado más de 25 puntos desde mediados de la década de los 90,
pasando de un 37% en 1995 hasta el 63% de 2020. Un dato nada
desdeñable pero todavía lejos de las tasas de países como Suecia
(83%), Estonia, Nueva Zelanda o Japón (79%). Una diferencia que se irá
acortando según la Comisión Europea, que estima que, con las políticas
adecuadas, la participación laboral de la poblac

In [29]:
doc2 = wrap(doc)

In [34]:
# As saw in the theory course, we start with the tokenization

lines = doc2.split('.')

lines

['La prolongación de la vida laboral mejora la sostenibilidad del estado\nde bienestar y del sistema público de pensiones, tiene externalidades\npositivas sobre el empleo y la productividad de los trabajadores\njóvenes y favorece el funcionamiento cognitivo',
 '  Estas son las\nprincipales conclusiones del informe de BBVA Research ‘Prolongar la\nvida laboral: ¿Por qué? ¿Dónde estamos?  ¿Cómo hacerlo?’, que analiza\nla evolución en la tasa de empleo de los mayores de 55 años y explica\nmedidas de éxito para impulsar la empleabilidad de este colectivo',
 '\r\nLa\ntasa de actividad de la población española entre 55 y 64 años se ha\nincrementado más de 25 puntos desde mediados de la década de los 90,\npasando de un 37% en 1995 hasta el 63% de 2020',
 ' Un dato nada\ndesdeñable pero todavía lejos de las tasas de países como Suecia\n(83%), Estonia, Nueva Zelanda o Japón (79%)',
 ' Una diferencia que se irá\nacortando según la Comisión Europea, que estima que, con las políticas\nadecuadas, la

In [35]:
len(lines)

25

In [36]:
# Looking at some documents, we see that sometimes '...' or such punctuation
# make the split messy (empty elements)

lines = [item for item in lines if item.strip()]

# item for item in lines : Parcourt chaque élément (item) dans la liste lines.
# if item.strip() : Filtre les éléments pour ne garder que ceux qui ne sont
# pas vides ou constitués uniquement d'espaces blancs (le .strip() supprime les
# espaces autour du texte et retourne une chaîne vide si le texte est
# uniquement des espaces).



In [37]:
len(lines)

24

In [38]:

lines

['La prolongación de la vida laboral mejora la sostenibilidad del estado\nde bienestar y del sistema público de pensiones, tiene externalidades\npositivas sobre el empleo y la productividad de los trabajadores\njóvenes y favorece el funcionamiento cognitivo',
 '  Estas son las\nprincipales conclusiones del informe de BBVA Research ‘Prolongar la\nvida laboral: ¿Por qué? ¿Dónde estamos?  ¿Cómo hacerlo?’, que analiza\nla evolución en la tasa de empleo de los mayores de 55 años y explica\nmedidas de éxito para impulsar la empleabilidad de este colectivo',
 '\r\nLa\ntasa de actividad de la población española entre 55 y 64 años se ha\nincrementado más de 25 puntos desde mediados de la década de los 90,\npasando de un 37% en 1995 hasta el 63% de 2020',
 ' Un dato nada\ndesdeñable pero todavía lejos de las tasas de países como Suecia\n(83%), Estonia, Nueva Zelanda o Japón (79%)',
 ' Una diferencia que se irá\nacortando según la Comisión Europea, que estima que, con las políticas\nadecuadas, la

In [40]:
vectorizer = TfidfVectorizer(stop_words=stopwords.words('spanish'), norm ='l1')

X = vectorizer.fit_transform(lines)

X #again, a matrix of same number of rows as lines and of columns = total number of words in the doc.

<24x250 sparse matrix of type '<class 'numpy.float64'>'
	with 361 stored elements in Compressed Sparse Row format>

In [42]:
# Simùilarities Matrix

S = cosine_similarity(X)
S
# Matrix of 24 rows and 24 columns (each row represent each row of Lines, and in
# each of these rows we have the cosine of this sentence with the 24 sentences
# (included itself))

# Note that the second cell of the first row is equal to the first cell of the
# second row (sentence 1 vs sentence2, sentence 2 vs sentence 1)

array([[1.        , 0.07489153, 0.        , 0.        , 0.01588207,
        0.        , 0.        , 0.04568102, 0.        , 0.        ,
        0.06304871, 0.04548424, 0.1813623 , 0.        , 0.3947188 ,
        0.28793237, 0.14325998, 0.04907773, 0.06200307, 0.02753899,
        0.        , 0.16741282, 0.        , 0.02038365],
       [0.07489153, 1.        , 0.09665427, 0.        , 0.06691155,
        0.04156691, 0.08137046, 0.14047156, 0.        , 0.        ,
        0.06402084, 0.03845585, 0.11759854, 0.        , 0.09394952,
        0.12659771, 0.04990897, 0.13866348, 0.02462607, 0.        ,
        0.10050448, 0.08578162, 0.        , 0.04684716],
       [0.        , 0.09665427, 1.        , 0.        , 0.17929728,
        0.        , 0.05698835, 0.16635917, 0.0957848 , 0.        ,
        0.        , 0.        , 0.        , 0.05647405, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.03253231],
       [0.   

In [46]:
S.shape

(24, 24)

In [51]:
S.sum(axis =1, keepdims=True)
#keepdims permet de garder la forme 24 lignes 1 colonne, plutot qu'une liste de 24 valeurs.

array([[2.57867728],
       [2.38882052],
       [1.68409024],
       [1.26863716],
       [2.00621806],
       [1.52291387],
       [1.4626499 ],
       [2.23036092],
       [1.85482558],
       [1.39670843],
       [1.95891943],
       [1.35505563],
       [1.91929658],
       [1.64769354],
       [2.29347409],
       [2.11698191],
       [1.6136381 ],
       [1.62436684],
       [1.815731  ],
       [1.66360574],
       [1.49188636],
       [2.04736828],
       [1.71036821],
       [1.68266465]])

In [52]:
S = S / (S.sum(axis = 1, keepdims=True))

In [53]:
S

array([[0.38779572, 0.02904261, 0.        , 0.        , 0.006159  ,
        0.        , 0.        , 0.0177149 , 0.        , 0.        ,
        0.02445002, 0.0176386 , 0.07033152, 0.        , 0.15307026,
        0.11165894, 0.05555561, 0.01903213, 0.02404452, 0.0106795 ,
        0.        , 0.06492197, 0.        , 0.00790469],
       [0.03135084, 0.41861663, 0.04046109, 0.        , 0.02801029,
        0.0174006 , 0.03406303, 0.05880373, 0.        , 0.        ,
        0.02680019, 0.01609826, 0.0492287 , 0.        , 0.03932883,
        0.05299591, 0.02089272, 0.05804684, 0.01030888, 0.        ,
        0.04207285, 0.03590961, 0.        , 0.019611  ],
       [0.        , 0.05739257, 0.59379241, 0.        , 0.10646537,
        0.        , 0.03383925, 0.09878281, 0.05687629, 0.        ,
        0.        , 0.        , 0.        , 0.03353386, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.01931744],
       [0.   

In [ ]:
# The next step should be the graph but we will not do it
# So now smoothing and regularization

# We can see this matrix as a transition matrix, because at the end we want to
# summary using sequence of sentences. If the value is 0 in one cell, mùeans
# that we just can't have the sequence with these 2 sentences (with proba 0)
# We want to have, at least a little proba that these 2 sentences follow each other

In [59]:
# Uniform Transition Matrix
U = np.ones_like(S) # shape of S but filled with ones
U

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 

In [60]:
# Let's normalize it same as S
U = U / (U.sum(axis = 1 , keepdims = True) )
U

array([[0.04166667, 0.04166667, 0.04166667, 0.04166667, 0.04166667,
        0.04166667, 0.04166667, 0.04166667, 0.04166667, 0.04166667,
        0.04166667, 0.04166667, 0.04166667, 0.04166667, 0.04166667,
        0.04166667, 0.04166667, 0.04166667, 0.04166667, 0.04166667,
        0.04166667, 0.04166667, 0.04166667, 0.04166667],
       [0.04166667, 0.04166667, 0.04166667, 0.04166667, 0.04166667,
        0.04166667, 0.04166667, 0.04166667, 0.04166667, 0.04166667,
        0.04166667, 0.04166667, 0.04166667, 0.04166667, 0.04166667,
        0.04166667, 0.04166667, 0.04166667, 0.04166667, 0.04166667,
        0.04166667, 0.04166667, 0.04166667, 0.04166667],
       [0.04166667, 0.04166667, 0.04166667, 0.04166667, 0.04166667,
        0.04166667, 0.04166667, 0.04166667, 0.04166667, 0.04166667,
        0.04166667, 0.04166667, 0.04166667, 0.04166667, 0.04166667,
        0.04166667, 0.04166667, 0.04166667, 0.04166667, 0.04166667,
        0.04166667, 0.04166667, 0.04166667, 0.04166667],
       [0.041

In [61]:
#Smoothed Similiarities Matrix
factor = 0.1 # arbitrary, but less than 1
S = (1 - factor)*S + factor*U
# most of the weight we be brought by S, but still U has a little part, adding
# the non null proba for every sequence possible

S


array([[0.35318281, 0.03030502, 0.00416667, 0.00416667, 0.00970977,
        0.00416667, 0.00416667, 0.02011008, 0.00416667, 0.00416667,
        0.02617169, 0.0200414 , 0.06746504, 0.00416667, 0.1419299 ,
        0.10465971, 0.05416671, 0.02129559, 0.02580674, 0.01377822,
        0.00416667, 0.06259644, 0.00416667, 0.01128089],
       [0.03238242, 0.38092164, 0.04058164, 0.00416667, 0.02937593,
        0.01982721, 0.03482339, 0.05709003, 0.00416667, 0.00416667,
        0.02828684, 0.0186551 , 0.0484725 , 0.00416667, 0.03956262,
        0.05186298, 0.02297012, 0.05640882, 0.01344466, 0.00416667,
        0.04203223, 0.03648532, 0.00416667, 0.02181657],
       [0.00416667, 0.05581998, 0.53857984, 0.00416667, 0.0999855 ,
        0.00416667, 0.03462199, 0.0930712 , 0.05535532, 0.00416667,
        0.00416667, 0.00416667, 0.00416667, 0.03434714, 0.00416667,
        0.00416667, 0.00416667, 0.00416667, 0.00416667, 0.00416667,
        0.00416667, 0.00416667, 0.00416667, 0.02155236],
       [0.004

In [65]:
# We can get back to the course page to remember how this works
# but in a nutshell, this matrix represent the links between tha graphs,
# and we also need the relative importance of each node (sentence) of the graph
# which is computed thanks to eigenvalue and eigenvectors. (1ere fois qu'on s'en sert)

eigenvals, eigenvecs = np.linalg.eig(S.T)#transposed

In [66]:
(eigenvals,eigenvecs)

(array([1.        , 0.18796554, 0.78171047, 0.71425166, 0.28628681,
        0.68357933, 0.67908061, 0.65052569, 0.62550451, 0.60464394,
        0.59056058, 0.33034892, 0.33782777, 0.55986228, 0.35756644,
        0.37319448, 0.39281993, 0.53728489, 0.52735859, 0.41456151,
        0.43215   , 0.44548041, 0.47133393, 0.48242036]),
 array([[ 2.65339427e-01,  7.71963461e-01, -2.42185514e-01,
         -1.56465159e-01,  4.91837342e-02, -2.99373449e-01,
          7.22097306e-02,  5.04782303e-02, -9.45274218e-02,
          6.86689052e-02, -3.67703279e-02, -2.24629482e-01,
          3.18099078e-01,  8.61432841e-02, -1.83525621e-02,
          1.76578054e-01,  8.27995537e-02,  1.35289269e-01,
          1.57163155e-01, -2.19599679e-02, -1.05605898e-01,
          3.83562397e-02,  2.99444728e-01, -1.61283846e-02],
        [ 2.53898272e-01,  8.69328701e-02, -1.25894256e-01,
         -2.16921060e-02,  6.34115913e-01,  7.89629218e-04,
          8.55763257e-02, -3.67677936e-02,  1.18245618e-01,
         

In [67]:
eigenvals # check the index of higher eigenvalue => index 0
# we take the associated vector of this index (eigenvec at index 0)

array([1.        , 0.18796554, 0.78171047, 0.71425166, 0.28628681,
       0.68357933, 0.67908061, 0.65052569, 0.62550451, 0.60464394,
       0.59056058, 0.33034892, 0.33782777, 0.55986228, 0.35756644,
       0.37319448, 0.39281993, 0.53728489, 0.52735859, 0.41456151,
       0.43215   , 0.44548041, 0.47133393, 0.48242036])

In [73]:
#eigenvecs[:, 0] : Sélectionne le vecteur propre d'indice 0 (c'est-à-dire la première colonne de la matrice des vecteurs propres).
#eigenvecs[0, :] : Sélectionne la première ligne de la matrice des vecteurs propres
eigenvecs[:,0]
# This now represent the 'value' or relative importance of each sentence.

array([0.26533943, 0.25389827, 0.19060748, 0.16634358, 0.22263582,
       0.17911182, 0.17342332, 0.2400425 , 0.21352957, 0.17326762,
       0.21286292, 0.16407231, 0.20489104, 0.19018336, 0.23747126,
       0.22324911, 0.18432478, 0.18178472, 0.20114954, 0.18963571,
       0.17799808, 0.21569723, 0.20092007, 0.19373724])

In [74]:
scores = eigenvecs[:,0]

In [75]:
sort_index = np.argsort(-scores)
sort_index

array([ 0,  1,  7, 14, 15,  4, 21,  8, 10, 12, 18, 22, 23,  2, 13, 19, 16,
       17,  5, 20,  6,  9,  3, 11])

In [76]:
print("Create summary:")
for i in sort_index[:5]:
  print(f' {scores[i], lines[i]}') # far better than the prvious project !!

Create summary:
 (0.26533942734813587, 'La prolongación de la vida laboral mejora la sostenibilidad del estado\nde bienestar y del sistema público de pensiones, tiene externalidades\npositivas sobre el empleo y la productividad de los trabajadores\njóvenes y favorece el funcionamiento cognitivo')
 (0.253898271568978, '  Estas son las\nprincipales conclusiones del informe de BBVA Research ‘Prolongar la\nvida laboral: ¿Por qué? ¿Dónde estamos?  ¿Cómo hacerlo?’, que analiza\nla evolución en la tasa de empleo de los mayores de 55 años y explica\nmedidas de éxito para impulsar la empleabilidad de este colectivo')
 (0.24004249961096635, '\nLa falta de incentivos para prolongar la vida laboral ha provocado que\nEspaña se sitúe entre las economías de la OCDE con una menor tasa de\nactividad de la población más mayor (entre 65 y 69 años)')
 (0.23747126468003932, '\nDe esta forma, prolongar la vida laboral de la fuerza de trabajo\nmejoraría la sostenibilidad del estado de bienestar y del sistema

In [77]:
lines

['La prolongación de la vida laboral mejora la sostenibilidad del estado\nde bienestar y del sistema público de pensiones, tiene externalidades\npositivas sobre el empleo y la productividad de los trabajadores\njóvenes y favorece el funcionamiento cognitivo',
 '  Estas son las\nprincipales conclusiones del informe de BBVA Research ‘Prolongar la\nvida laboral: ¿Por qué? ¿Dónde estamos?  ¿Cómo hacerlo?’, que analiza\nla evolución en la tasa de empleo de los mayores de 55 años y explica\nmedidas de éxito para impulsar la empleabilidad de este colectivo',
 '\r\nLa\ntasa de actividad de la población española entre 55 y 64 años se ha\nincrementado más de 25 puntos desde mediados de la década de los 90,\npasando de un 37% en 1995 hasta el 63% de 2020',
 ' Un dato nada\ndesdeñable pero todavía lejos de las tasas de países como Suecia\n(83%), Estonia, Nueva Zelanda o Japón (79%)',
 ' Una diferencia que se irá\nacortando según la Comisión Europea, que estima que, con las políticas\nadecuadas, la